In [30]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import random
%matplotlib inline
import re
import warnings

In [31]:
# success rate of gpu's
# what does job termination mean - lose work? means that calculating time is important
#requested jobs used time and terminated -why a buffer
#......................................................
#failures by user - who and not completeing - using a lot of resources 
#failed jobs - bad users or hardware failure (exitcode)
#envoriment side  -- first value user - exitcode, second environment 

#'0:9 and 0:15 are the most common exit codes for jobs with some version of the state of "CANCELLED"


In [32]:
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [33]:
accre = pd.read_csv('accre-gpu-jobs-2022-v2.csv', on_bad_lines='skip')
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [34]:
accre = pd.DataFrame(accre)
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [35]:
accre['USEDMEM'] = accre['USEDMEM'].str.replace(r'[a-zA-Z]','')
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [36]:
accre['REQTIME'] = accre['REQTIME'].astype(str)
accre['USEDTIME'] = accre['USEDTIME'].astype(str)
accre['REQTIME'] = pd.to_timedelta(accre['REQTIME'].str.replace('-',' days '))
accre['USEDTIME'] = pd.to_timedelta(accre['USEDTIME'].str.replace('-',' days '))
accre['REQTIME'] = pd.to_timedelta(accre['REQTIME']).dt.total_seconds()
accre['USEDTIME'] = pd.to_timedelta(accre['USEDTIME']).dt.total_seconds()
accre = accre[accre['USEDMEM'].map(len) > 1]
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,172800.0,793.0,1,2,1,pascal,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,18000.0,537.0,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,72000.0,18586.0,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,54000.0,15070.0,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40,302400.0,67776.0,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,432000.0,52.0,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70,432000.0,2091.0,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21,432000.0,243.0,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52,432000.0,673.0,1,1,1,turing,0:0,COMPLETED


In [104]:
accre_glasshouse = accre[accre['ACCOUNT'] == 'glasshouse']
accre_virginia = accre[accre['ACCOUNT'] == 'virginia']
accre_malanga = accre[accre['ACCOUNT'] == 'malanga']
accre_prince = accre[accre['ACCOUNT'] == 'prince']
accre_prince = accre[accre['ACCOUNT'] == 'prince']
accre_grisette = accre[accre['ACCOUNT'] == 'grisette']
accre_harovin = accre[accre['ACCOUNT'] == 'harovin']
accre_antares = accre[accre['ACCOUNT'] == 'antares']
accre_sunburst = accre[accre['ACCOUNT'] == 'sunburst']
accre_pasilla = accre[accre['ACCOUNT'] == 'pasilla']
accre_plumcot = accre[accre['ACCOUNT'] == 'plumcot']
accre_pecans = accre[accre['ACCOUNT'] == 'pecans']
accre_kinnow = accre[accre['ACCOUNT'] == 'kinnow']
accre_bertanne = accre[accre['ACCOUNT'] == 'bertanne']
accre_almonds = accre[accre['ACCOUNT'] == 'almonds']
accre_apple = accre[accre['ACCOUNT'] == 'apple']
accre_cactus = accre[accre['ACCOUNT'] == 'cactus']

accre_sunburst

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
65,32880916,sunburst,demarcus,151.40,345600.0,333.0,1,1,1,pascal,0:0,COMPLETED
66,32880917,sunburst,demarcus,143.45,345600.0,328.0,1,1,1,pascal,0:0,COMPLETED
67,32880918,sunburst,demarcus,140.25,345600.0,293.0,1,1,1,pascal,0:0,COMPLETED
68,32880919,sunburst,demarcus,156.66,345600.0,2441.0,1,1,1,pascal,0:0,COMPLETED
69,32880920,sunburst,demarcus,173.94,345600.0,4136.0,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
333426,46328877,sunburst,demarcus,238.02,345600.0,85496.0,1,2,2,pascal,0:0,COMPLETED
334421,46512880,sunburst,demarcus,430.72,345600.0,217369.0,1,2,2,pascal,0:0,COMPLETED
334422,46512881,sunburst,demarcus,419.10,345600.0,210937.0,1,2,2,pascal,0:0,COMPLETED
334423,46512882,sunburst,demarcus,414.15,345600.0,216998.0,1,2,2,pascal,0:0,COMPLETED


In [66]:
accre_glasshouse_failed_count = accre_glasshouse[accre_glasshouse['STATE'] == 'FAILED']
accre_glasshouse_failed_count.info()
accre_glasshouse_failed_percentage = 250/234998


In [71]:
accre_virginia_failed_count = accre_virginia[accre_virginia['STATE'] == 'FAILED']
accre_virginia_failed_count.info()
accre_virginia_failed_percentage = 0/9488

In [80]:
accre_malanga_failed_count = accre_malanga[accre_malanga['STATE'] == 'FAILED']
#accre_malanga_failed_count.info()
accre_malanga_failed_percentage = 0/8157

In [89]:
accre_prince_failed_count = accre_prince[accre_prince['STATE'] == 'FAILED']
#accre_prince_failed_count
accre_prince_failed_percentage = 0/7174

In [ ]:
accre_malanga_failed_count = accre_malanga[accre_malanga['STATE'] == 'FAILED']
#accre_malanga_failed_count.info()
accre_malanga_failed_percentage = 0/8157

In [94]:
accre_grisette_failed_count = accre_grisette[accre_grisette['STATE'] == 'FAILED']
#accre_grisette_failed_count.info()
accre_grisette_failed_percentage = 1/5242

In [98]:
accre_harovin_failed_count = accre_harovin[accre_harovin['STATE'] == 'FAILED']
#accre_harovin_failed_count.info()
accre_harovin_failed_percentage = 4/4534

In [103]:
accre_antares_failed_count = accre_antares[accre_antares['STATE'] == 'FAILED']
#accre_antares_failed_count.info()
accre_antares_failed_percentage = 0/3015

In [107]:
accre_sunburst_failed_count = accre_sunburst[accre_sunburst['STATE'] == 'FAILED']
#accre_sunburst_failed_count.info()
accre_sunburst_failed_percentage = 87/1505

In [113]:
accre_pasilla_failed_count = accre_pasilla[accre_pasilla['STATE'] == 'FAILED']
#accre_pasilla_failed_count.info()
accre_pasilla_failed_percentage = 1/1263

In [116]:
accre_plumcot_failed_count = accre_plumcot[accre_plumcot['STATE'] == 'FAILED']
#accre_plumcot_failed_count.info()
accre_plumcot_failed_percentage = 0/790

In [121]:
accre_pecans_failed_count = accre_pecans[accre_pecans['STATE'] == 'FAILED']
#accre_pecans_failed_count.info()
accre_pecans_failed_percentage = 0/576

In [123]:
accre_kinnow_failed_count = accre_kinnow[accre_kinnow['STATE'] == 'FAILED']
#accre_kinnow_failed_count.info()
accre_kinnow_failed_percentage = 0/406

In [126]:
accre_bertanne_failed_count = accre_bertanne[accre_bertanne['STATE'] == 'FAILED']
#accre_bertanne_failed_count.info()
accre_bertanne_failed_percentage = 1/198

In [133]:
accre_almonds_failed_count = accre_almonds[accre_almonds['STATE'] == 'FAILED']
#accre_almonds_failed_count.info()
accre_almonds_failed_percentage = 6/197

In [139]:
accre_apple_failed_count = accre_apple[accre_apple['STATE'] == 'FAILED']
#accre_apple_failed_count.info()
accre_apple_failed_percentage = 11/186

In [141]:
accre_cactus_failed_count = accre_cactus[accre_cactus['STATE'] == 'FAILED']
#accre_cactus_failed_count.info()
accre_cactus_failed_percentage = 0/113

In [ ]:
 glasshouse       234998
virginia           9488
malanga            8157
prince             7174
grisette           5242
harovin            4534
antares            3015
sunburst           1505
pasilla            1263
plumcot             790
pecans              576
kinnow              406
bertanne            198
almonds             197
apple               186
cactus              113)

In [ ]:
#Looking at Accounts Failed rate and who's fault#/distribution/Users from accounts-is there a individual who is failing/
#By what percent are they off/Do users get better with jobs-corr?

In [10]:
accre_pascal_state

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
1247,32896784,glasshouse,miya,19.59,00:30:00,00:09:54,1,1,1,pascal,127:0,FAILED
9370,32940928,glasshouse,arlyne,16331.96,1-06:00:00,1-06:00:16,7,7,32,pascal,1:0,FAILED
14105,33016633,sunburst,demarcus,0,4-00:00:00,00:00:15,1,1,1,pascal,1:0,FAILED
14117,33016701,sunburst,demarcus,0,4-00:00:00,00:00:01,1,1,1,pascal,1:0,FAILED
14119,33016901,sunburst,demarcus,0,4-00:00:00,00:00:02,1,1,1,pascal,1:0,FAILED
...,...,...,...,...,...,...,...,...,...,...,...,...
332515,46179924,glasshouse,lea,52415.96,5-00:00:00,3-01:57:43,1,7,8,pascal,1:0,FAILED
332517,46179927,glasshouse,lea,13839.69,5-00:00:00,3-01:54:36,1,7,8,pascal,1:0,FAILED
332746,46287083,glasshouse,tommie,82628.63,3-18:00:00,00:25:12,2,8,8,pascal,1:0,FAILED
334706,46525496,glasshouse,mel,11823.83,5-00:00:00,2-02:00:25,3,12,16,pascal,1:0,FAILED


In [8]:
accre_pascal = accre_df[accre_df['PARTITION'] == 'pascal']
accre_maxwell = accre_df[accre_df['PARTITION'] == 'maxwell']
accre_turing = accre_df[accre_df['PARTITION'] == 'turing']

In [ ]:
fig, ax = plt.subplots(1, 3,figsize = (20,13))


sns.distplot(pascal_usedmem, hist=True, bins = 10, color = 'orange', ax = ax[0]),
sns.distplot(pascal_usedtime, hist=True, bins = 10, color = 'black', ax = ax[0]),
sns.distplot(maxwell_usedmem, hist=False, bins = 5, color = 'orange', ax = ax[1]),
sns.distplot(maxwell_usedtime, hist=False, bins = 5, color = 'black', ax = ax[1]),
sns.distplot(turing_usedmem, hist=False, bins = 5, color = 'orange', ax = ax[2]),
sns.distplot(turing_usedtime, hist=False, bins = 5, color = 'black', ax = ax[2])

plt.show()

In [ ]:
asking for enough memory/types of failures
asking too much memory

In [ ]:
# .squeeze() converts df to series
#accre_list = pd.DataFrame(accre, columns=['USEDMEM'])
#accre_list = accre_list.squeeze()
